## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-09-16-54-57 +0000,bbc,Starmer praises Trump over role in Gaza ceasef...,https://www.bbc.com/news/articles/c33rgd5gmrxo...
1,2025-10-09-16-54-30 +0000,missionlocal,Pelosi challenger Saikat Chakrabarti launches ...,https://missionlocal.org/2025/10/pelosi-challe...
2,2025-10-09-16-52-54 +0000,nyt,Live Updates: Trump Calls Gaza Deal a ‘Momento...,https://www.nytimes.com/live/2025/10/09/world/...
3,2025-10-09-16-51-13 +0000,nypost,California becomes first state to phase out ul...,https://nypost.com/2025/10/09/us-news/californ...
4,2025-10-09-16-50-34 +0000,nyt,Another Guilty Verdict After Appeal in Pelicot...,https://www.nytimes.com/2025/10/09/pelicot-rap...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2294/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,51
6,deal,32
4,gaza,29
149,new,22
92,hamas,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
237,2025-10-08-23-17-50 +0000,nypost,Trump announces Israel and Hamas signed off on...,https://nypost.com/2025/10/08/world-news/trump...,205
256,2025-10-08-22-30-24 +0000,wapo,"Trump, Hamas officials say Gaza ceasefire deal...",https://www.washingtonpost.com/world/2025/10/0...,147
13,2025-10-09-16-41-03 +0000,wapo,Live updates: Israel and Hamas sign first phas...,https://www.washingtonpost.com/world/2025/10/0...,147
121,2025-10-09-11-51-45 +0000,nypost,Gazans join Israelis in chanting Trump’s name ...,https://nypost.com/2025/10/09/us-news/gazans-j...,142
334,2025-10-08-18-05-37 +0000,nypost,"Trump mulls weekend trip to Egypt, Gaza to sea...",https://nypost.com/2025/10/08/world-news/israe...,139


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
237,205,2025-10-08-23-17-50 +0000,nypost,Trump announces Israel and Hamas signed off on...,https://nypost.com/2025/10/08/world-news/trump...
240,71,2025-10-08-23-12-49 +0000,latimes,"After Palisades fire failures, L.A. promises t...",https://www.latimes.com/california/story/2025-...
143,69,2025-10-09-10-39-00 +0000,wsj,China introduced new controls on rare-earth ex...,https://www.wsj.com/economy/trade/china-impose...
250,49,2025-10-08-22-51-00 +0000,wsj,President Trump’s declaration that Illinois Go...,https://www.wsj.com/politics/policy/trump-prit...
273,43,2025-10-08-21-33-46 +0000,nypost,Andrew Cuomo would scrap plan to close Rikers ...,https://nypost.com/2025/10/08/us-news/andrew-c...
313,42,2025-10-08-19-34-04 +0000,nypost,House Speaker Mike Johnson rejects standalone ...,https://nypost.com/2025/10/08/us-news/house-sp...
248,39,2025-10-08-23-00-00 +0000,wsj,"In his second term, Trump has taken control of...",https://www.wsj.com/politics/policy/trump-doj-...
228,38,2025-10-08-23-41-55 +0000,nypost,California gubernatorial candidate Katie Porte...,https://nypost.com/2025/10/08/us-news/katie-po...
12,35,2025-10-09-16-41-31 +0000,bbc,Texas court blocks death row inmate's executio...,https://www.bbc.com/news/articles/cn4lexk7wn7o...
330,34,2025-10-08-18-40-56 +0000,nypost,Most Fed officials supported further rate cuts...,https://nypost.com/2025/10/08/business/most-fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
